In [ ]:
import os
import tensorflow as tf
import numpy as np
import logging

EPOCH = 10
BATCH_SIZE = 1000
TOTAL_BATCH = 10

IMAGE_WIDTH = 100
IMAGE_HEIGHT = 100

# prepare dataset
filename = tf.train.match_filenames_once("./data/train/*.jpg");
filename_queue = tf.train.string_input_producer(filename)

image_reader = tf.WholeFileReader()
image_name, image_file = image_reader.read(filename_queue)

image = tf.image.decode_jpeg(image_file, channels=3)
image = tf.cast(image, tf.float32)
resized_image = tf.image.resize_images(image, [IMAGE_WIDTH, IMAGE_HEIGHT])

resized_image = tf.train.batch([resized_image], batch_size=BATCH_SIZE)
image_name = tf.train.batch([image_name], batch_size=BATCH_SIZE)

# make cnn model
x = tf.placeholder(tf.float32, [None, IMAGE_WIDTH, IMAGE_HEIGHT, 3])
y = tf.placeholder(tf.float32, [None, 2])
keep_prob = tf.placeholder(tf.float32)

# x = resized_image
# y = image_name
# keep_prob = 0.8

# w1 [5 5 3 32] -> [5 5]: 커널 크기, 3: 입력값 X 의 특성수, 32: 필터 갯수
# out1 Conv shape=(?, 100, 100, 32)
#      Pool     ->(?, 50, 50, 32)
w1 = tf.Variable(tf.random_normal([5, 5, 3, 32], stddev=0.01))
b1 = tf.Variable(tf.zeros([32]))
conv1 = tf.nn.conv2d(x, w1, strides=[1, 1, 1, 1], padding='SAME')
h1 = tf.nn.relu(conv1 + b1)
out1 = tf.nn.max_pool(h1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

# out2 Conv shape=(?, 50, 50, 64)
#      Pool     ->(?, 25, 25, 64)
w2 = tf.Variable(tf.random_normal([3, 3, 32, 64], stddev=0.01))
b2 = tf.Variable(tf.zeros([64]))
conv2 = tf.nn.conv2d(out1, w2, strides=[1, 1, 1, 1], padding='SAME')
h2 = tf.nn.relu(conv2)
out2 = tf.nn.max_pool(h2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

# FC 레이어: 입력값 25x25x64 -> 출력값 40000
# Full connect를 위해 직전의 Pool 사이즈인 (?, 25, 25, 64) 를 참고하여 차원을 줄인다
#    Reshape  ->(?, 256)
w3 = tf.Variable(tf.random_normal([25 * 25 * 64, 500], stddev=0.01))
r3 = tf.reshape(out2, [-1, 25 * 25 * 64])
h3 = tf.matmul(r3, w3)
h3 = tf.nn.relu(h3)
h3 = tf.nn.dropout(h3, keep_prob)

w4 = tf.Variable(tf.random_normal([500, 2], stddev=0.01))
model = tf.matmul(h3, w4)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=model, labels=y))
optimizer = tf.train.AdamOptimizer(0.001).minimize(cost)
#


with tf.Session() as sess:
    init_op = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())
    sess.run(init_op)
    
    # open filename queue
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)
    
    for epoch in range(EPOCH):
        total_cost = 0
        for i in range(TOTAL_BATCH):
            try:

                datas = sess.run(resized_image)
                labels = sess.run(image_name)
                labels_input = [[0, 1] if 'cat' in label.decode('utf8') else [1, 0] for label in labels]
#                 print('Label: ', labels)
#                 print('Label input:', labels_input)
                
                ans = sess.run(model, feed_dict={x: datas, y:labels_input, keep_prob:0.8})
                _, cost_val = sess.run([optimizer, cost], feed_dict={x: datas, y:labels_input, keep_prob:0.8})
#                 print('Ans:', ans)
#                 print('cost_val: ', cost_val)
                
                total_cost += cost_val

            except Exception as e:
                logging.exception("message")
                continue
                
        print('total_cost: ', total_cost)
    
    # close filename queue
    coord.request_stop()
    coord.join(threads)

total_cost:  32.9166442752
total_cost:  6.93250513077
total_cost:  6.93153685331
